In [223]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest,chi2
import pickle



In [179]:
df = pd.read_csv('/content/train (1).csv')

In [180]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [181]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [182]:
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [183]:
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=42)

In [198]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((712, 7), (179, 7), (712,), (179,))

In [206]:
ct1 = ColumnTransformer(transformers=[
    ('si_age',SimpleImputer(),['Age']),
    ('si_embarked',SimpleImputer(strategy='most_frequent'),['embarked'])],remainder='passthrough')



In [214]:
ct2 = ColumnTransformer(transformers=[
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),['Sex','Embarked'])],remainder='passthrough')

In [220]:
ct3 = ColumnTransformer(transformers=[('scaling',MinMaxScaler(),slice(0,10))])

In [224]:
ct4 = SelectKBest(score_func=chi2,k=8)

In [225]:
ct5 = DecisionTreeClassifier()

In [226]:
pipe = Pipeline([
    ('ct1',ct1),
    ('ct2',ct2),
    ('ct3',ct3),
    ('ct4',ct4),
    ('ct5',ct5)
])


In [231]:
pipe

Pipeline(steps=[('ct1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('si_age', SimpleImputer(),
                                                  ['Age']),
                                                 ('si_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['embarked'])])),
                ('ct2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['Sex', 'Embarked'])])),
                ('ct3',
                 ColumnTransformer(transformers=[('scaling', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('ct4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7b3b02931080>)),
                ('ct5', DecisionTreeClassifier())])

In [233]:
pipe.fit(x_train,y_train)

ValueError: A given column is not a column of the dataframe